# Tratamento dos Dados Questão 2

Com o intuito de responder à segunda questão de pesquisa, foi realizado o carregamento do dataset e analisadas as informações dos arquivos de configuração para identificar quais são os plugins utilizados. Em seguida, os plugins foram extraídos das dependências e essas informações foram mescladas com os dados do dataset.

## Importação de Bibliotecas

- **pandas (`pd`)**: Para manipulação de dados em formato tabular.
- **json**: Trabalhar com dados no formato JSON
- **matplotlib.pyplot (`plt`)**: Para visualização de gráficos.
- **os**: Para interagir com o sistema operacional, como criação de diretórios.
- **seaborn (`sns`)**: Para visualização de gráficos estatísticos.

## Funcionalidades

1. **Carregamento de Dados**: O script carrega dois arquivos CSV:
   - Um arquivo que contém informações sobre pacotes (presumivelmente no formato `package.json`).
   - Um segundo arquivo que contém informações sobre repositórios.

2. **Extração de Plugins**: Para cada linha do primeiro arquivo:
   - Tenta carregar o conteúdo da coluna `content` como um objeto JSON.
   - Extrai o nome do repositório e as dependências (tanto `dependencies` quanto `devDependencies`).
   - Combina todas as dependências em uma lista e armazena as informações em uma lista de dicionários.

3. **Criação de DataFrame**: Converte a lista de dicionários em um DataFrame do `pandas` para facilitar a visualização e análise.

4. **Mesclagem de Dados**: Mescla o DataFrame de plugins de repositórios com o DataFrame de repositórios com base na coluna `name`, utilizando uma junção interna.

5. **Geração de Relatórios**: Salva os DataFrames resultantes em arquivos CSV para análise futura e exibe os resultados.


In [5]:
import pandas as pd
import json

# Carregar o arquivo package.json.csv
# Substitua pelo caminho correto
file_path = '../Database/Questao1/separados/package.json.csv'
data = pd.read_csv(file_path)

# Carregar os dois arquivos CSV
file_path1 = '../Database/Resultados-Busca/repositories.csv'  # Substitua pelo caminho correto do primeiro arquivo
data1 = pd.read_csv(file_path1)

# Lista para armazenar informações de cada repositório e seus plugins
repositories_plugins = []

# Processar cada linha da coluna 'content' como JSON
for index, row in data.iterrows():
    try:
        # Tentar carregar cada linha como JSON
        package_data = json.loads(row['content'])

        # Extrair informações do repositório
        repo_name = row['repository']  # Substitua pelo nome correto da coluna, se necessário

        # Extrair plugins das seções 'dependencies' e 'devDependencies'
        dependencies = list(package_data.get('dependencies', {}).keys())
        dev_dependencies = list(package_data.get('devDependencies', {}).keys())
        
        # Combinar todas as dependências em uma lista
        all_plugins = dependencies + dev_dependencies

        # Adicionar entrada para o repositório atual
        repositories_plugins.append({
            'name': repo_name,
            'Plugins': all_plugins
        })

    except json.JSONDecodeError as e:
        print(f"Erro ao carregar JSON na linha {index}: {e}")

# Converter para DataFrame para visualizar os plugins de cada repositório
repositories_plugins_df = pd.DataFrame(repositories_plugins)

# Salvar em um arquivo CSV para análise
repositories_plugins_df.to_csv('../Database/Questao2/repository_plugins.csv', index=False)

# Exibir a lista de repositórios com seus plugins
print(repositories_plugins_df)

# Concatenar as duas tabelas com base na coluna 'name'
merged_data = pd.merge(data1, repositories_plugins_df, on='name', how='inner')  # 'inner' mantém apenas as correspondências

# Salvar o resultado em um novo arquivo CSV para análise
merged_data.to_csv('../Database/Questao2/merged_data.csv', index=False)

# Exibir a tabela concatenada
print(merged_data)

                                  name  \
0                          XiaoMi/hiui   
1                ant-design/ant-design   
2                         Tencent/weui   
3              tailwindlabs/headlessui   
4          vueComponent/ant-design-vue   
..                                 ...   
690            swisspost/design-system   
691              kondasoft/ks-bootshop   
692           supermedium/aframe-react   
693  ngokevin/aframe-react-boilerplate   
694   transferthought/transfer-thought   

                                               Plugins  
0    [@babel/plugin-external-helpers, @babel/plugin...  
1    [@ant-design/colors, @ant-design/cssinjs, @ant...  
2    [autoprefixer, browser-sync, gulp, gulp-conver...  
3    [@arethetypeswrong/cli, @swc-node/register, @s...  
4    [@ant-design/colors, @ant-design/icons-vue, @b...  
..                                                 ...  
690           [@changesets/cli, start-server-and-test]  
691  [bootstrap, swiper, autoprefixer, 

# Correlação entre Contribuidores, Estrelas e Plugins

O objetivo é comparar a presença de plugins em projetos UX/UI e não UX/UI, gerando heatmaps que mostram a distribuição de bibliotecas com base em faixas de estrelas e contribuidores.

## Funcionalidades

1. **Criação de Diretório**: O script cria um diretório para salvar as imagens geradas, caso ele não exista.

2. **Carregamento de Dados**: Carrega dois arquivos CSV:
   - Um arquivo com dados filtrados de repositórios.
   - Um segundo arquivo com informações sobre projetos, incluindo se são UX/UI.

3. **Separação de Projetos**: Filtra os projetos em duas categorias:
   - **Projetos UX/UI**: Projetos que têm o valor `1` na coluna "É UI?".
   - **Projetos Não UX/UI**: Projetos que têm o valor `0` na coluna "É UI?".

4. **Filtragem de Plugins**: 
   - Extrai uma lista de plugins desejados dos projetos UX/UI.
   - Converte a coluna de plugins em listas de strings e filtra apenas os plugins que estão na lista desejada.

5. **Criação de Colunas de Plugins**: Para cada plugin único, cria uma nova coluna no DataFrame que indica a presença ou ausência do plugin.

6. **Geração de Heatmaps**: 
   - Cria heatmaps para visualizar a presença de bibliotecas com base em categorias de estrelas e contribuidores.
   - Usa a função `plot_and_save_heatmap` para gerar e salvar os gráficos.

7. **Agrupamento de Dados**: Agrupa os dados por categorias de estrelas e contribuidores, somando a presença dos plugins.

In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt

# Cria o diretório para salvar as imagens, se não existir
output_dir = '../Image/Questao2/comparacao'
os.makedirs(output_dir, exist_ok=True)

# Carrega os CSVs
data = pd.read_csv('../Database/Questao2/merged_data.csv')
df = pd.read_csv('../Database/Questao1/Final/pesquisa-mais-frequentes.csv')

# Separar UX/UI e não UX/UI
ui_projects = df[df["É UI?"] == 1]
non_ui_projects = df[df["É UI?"] == 0]

# Exibir resultados
print("Projetos UX/UI:")
print(ui_projects)
print("\nProjetos Não UX/UI:")
print(non_ui_projects)

# Seleciona apenas as colunas desejadas
colunas_desejadas = [
    "name", "description", "stars", "language", "forks", "url",
    "created_at", "updated_at", "open_issues", "default_branch",
    "contributors_count", "commits_count", "topics", "watchers",
    "size", "category", "Plugins"
]
data_filtrado = data[colunas_desejadas]

# Lista de plugins específicos que você quer extrair
plugins_desejados = ui_projects['plugins-ui'].unique()

# Converter a coluna 'Plugins' para listas de strings
data_filtrado['Plugins'] = data_filtrado['Plugins'].apply(lambda x: x.strip("[]").replace("'", "").split(", ") if isinstance(x, str) else [])

# Filtrar e extrair apenas os plugins desejados
plugins_filtrados = set(
    plugin for plugins in data_filtrado['Plugins'] for plugin in plugins if plugin in plugins_desejados
)

# Exibir os plugins filtrados
print(plugins_filtrados)

# Criar uma nova coluna para cada plugin único
for plugin in plugins_filtrados:
    data_filtrado[plugin] = data_filtrado['Plugins'].apply(lambda x: 1 if plugin in x else 0)

# Função para gerar e salvar heatmaps
def plot_and_save_heatmap(data, title, xlabel, ylabel, filename):
    plt.figure(figsize=(12, 8))  # Aumentando o tamanho da figura
    sns.heatmap(data, annot=True, fmt="d", cmap="Blues")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=45, ha='right')  # Rotacionando as legendas do eixo x
    plt.tight_layout()  # Ajustando o layout
    plt.savefig(os.path.join(output_dir, filename))
    plt.close()

# Adicionando categorias para faixas de estrelas
data_filtrado['stars_category'] = pd.cut(data_filtrado['stars'], bins=[0, 10, 100, 1000, data_filtrado['stars'].max()], labels=['<10', '10-100', '100-1000', '>1000'])

# Agrupando por faixa de estrelas
bibs_by_stars = data_filtrado.groupby('stars_category')[list(plugins_filtrados)].sum()
plot_and_save_heatmap(bibs_by_stars, "Presença de Bibliotecas por Faixa de Estrelas", "Biblioteca", "Faixa de Estrelas", "heatmap_stars.png")

# Adicionando categorias para faixas de contribuidores
data_filtrado['contributors_category'] = pd.cut(data_filtrado['contributors_count'], bins=[0, 10, 50, 100, data_filtrado['contributors_count'].max()], labels=['<10', '10-50', '50-100', '>100'])

# Agrupando por faixa de contribuidores
bibs_by_contributors = data_filtrado.groupby('contributors_category')[list(plugins_filtrados)].sum()
plot_and_save_heatmap(bibs_by_contributors, "Presença de Bibliotecas por Faixa de Contribuidores", "Biblioteca", "Faixa de Contribuidores", "heatmap_contributors.png")

Projetos UX/UI:
                     plugins-ui  Frequência  \
3                         react         287   
4                     react-dom         219   
7                  @types/react         172   
20                 autoprefixer         116   
23                      postcss         101   
24             @types/react-dom          99   
28          react-test-renderer          94   
29                       rollup          91   
33       @testing-library/react          80   
35                         vite          79   
36                          vue          79   
38                   css-loader          77   
39                 react-native          76   
40                         sass          75   
41    @testing-library/jest-dom          74   
47                  tailwindcss          62   
48                   prop-types          62   
56                        chalk          55   
63                    stylelint          53   
66                 style-loader          51 

C:\Users\tochi\AppData\Local\Temp\ipykernel_9332\349650002.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtrado['Plugins'] = data_filtrado['Plugins'].apply(lambda x: x.strip("[]").replace("'", "").split(", ") if isinstance(x, str) else [])
C:\Users\tochi\AppData\Local\Temp\ipykernel_9332\349650002.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtrado[plugin] = data_filtrado['Plugins'].apply(lambda x: 1 if plugin in x else 0)
C:\Users\tochi\AppData\Local\Temp\ipykernel_9332\3496500